In [ ]:
import re
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn

seaborn.set_style("whitegrid")

dfs = []

for fp in snakemake.input["results"][::-1]:

    # Extract "pgpfactor" value from file path
    pgpfactor = re.findall("\/pgpfactor_(\d+\.?\d*)\/", fp)
    assert len(pgpfactor) == 1
    pgpfactor = float(pgpfactor[0])

    # Read single scenario values and combine later
    data = pd.read_csv(fp)
    data["pgpfactor"] = pgpfactor  # Used as index / x-axis later
    dfs.append(data)

# Create single df with all results in correct format for plotting
df = pd.concat(dfs)
df = df.pivot(index="pgpfactor", columns="variable", values="value")

# Variables to plot and order (in legend from bottom to top)
plot_order = [
    "Natural gas",
    "Natural gas with CCS",
    "Solar",
    "Wind",
    "Battery storage",
    "Unmet demand",
    "Advanced nuclear",
    "TES plus extra generator",
    "PGP",
]

# Colors in order for each technology
color_order = [snakemake.config["plotting"]["colors"][v] for v in plot_order]

# Select variables to plot and order
df = df[plot_order]

df.plot.area(color=color_order, figsize=(8, 3), lw=0)

plt.xlabel("Fraction of base case cost")
plt.ylabel("System cost [USD/kWh]")
plt.title(
    f"{snakemake.wildcards['template']} /  "
    f"{snakemake.wildcards['technologydata']} scenario for "
    f"{snakemake.wildcards['country']} ({snakemake.wildcards['years']})"
)

plt.xlim(0.1, 1)
plt.ylim(0, 0.1)

plt.xticks(np.linspace(0.1, 1, 10))

# Reverse legend sort order and place to right of plot
ax = plt.gca()
handles, labels = ax.get_legend_handles_labels()
ax.legend(handles[::-1], labels[::-1], bbox_to_anchor=(1, 1))

for fn in snakemake.output["figure"]:
    plt.savefig(fn, dpi=300, bbox_inches="tight")